This is a way to combine multiple parameters for use in a single for loop where you need to use every combination of all parameters.

For example if we want: 
- Mass between 0.1 and 2 $M_\odot$ in steps of 0.1 $M_\odot$ 
- $T_{eff}$  between 2000 K and 5000 K in steps of 500 K
- $log(g)$ between 3 and 5 in steps of 0.05
- $Z$ between -1.5 and 0.5 in steps of 0.1

Then we have many combinations and looping around these in nested loops is in practical

For example:

In [5]:
# imports
import numpy as np

# define ranges of parameters
Mass = np.arange(0.1, 2.1, 0.1)
Teff = np.arange(2000.0, 5500.0, 500.0)
logg = np.arange(3, 5.1, 0.05)
Z = np.arange(-1.5, 0.6, 0.1)

# storage arrays
combinations = []
results = []
# Create all combinations and work out result
for m in Mass:
    for t in Teff:
        for g in logg:
            for z in Z:
                # store combinations
                combinations.append([m, t, g, z])
                # calcualte this iteration result
                result = m + t**(-3) + g/z
                # store result
                results.append(result)

This 4 nested "for" loops already looks ugly, for many parameters this just becomes unfeasible.

However there is a better way using itertools:

In [8]:
#imports
import numpy as np
import itertools

# define ranges of parameters
Mass = np.arange(0.1, 2.1, 0.1)
Teff = np.arange(2000.0, 5500.0, 500.0)
logg = np.arange(3, 5.1, 0.05)
Z = np.arange(-1.5, 0.6, 0.1)

# get all combinations as a 1D list
combinations = list(itertools.product(Mass, Teff, logg, Z))

Note here that I use "list(x)" to convert the itertools generator to a list. A generator will not have any access to values until they are used in, for example, a "for" loop. We will want to keep track of the combinations so we must create the whole list.

Now we can do our single for loop:

In [9]:
# storage
results = []
# single for loop around each variable
for combination in combinations:
    # get variables for this iteration
    Mass_i, Teff_i, logg_i, Z_i = combination
    # calcualte this iteration result
    result = Mass_i + Teff_i**(-3) + logg_i/Z_i
    # store result
    results.append(result)

Note that each combinbation in "combinations" is stored as a tuple or list of the invidiual parameters in the **same** order as placed in the itertools.product command.

As such there is a neat way to compress the above code by defining a function to do the calculate the result:

In [11]:
# define worker function that calculates result from input parameters
#  Note these must be in the same order as input to itertools.product
def science_result(mass, teff, lg, z):
    result = mass + teff**(-3) + lg/z
    # return result
    return result
# storage
results = []
# compressed for loop
for combination in combinations:
    # work out result directly
    result = science_result(*combination)
    # store result
    results.append(result)

Note in our example above we use the asterix notation, this simply "pipes" the tuple or list variables into the variables called in the function (and as such **MUST** be the same length as number of arguments in our function (in our case 4). If combinations was correctly created using our 4 variables we know that each combination will have 4 values for each iteration, thus this "piping" works as one would expect.

One further thing we can do with this is note that actually we don't need to for loop at all in this situation.

Using numpy matrix operations we can actually workout all combinations of the "science result" at once (utalising numpys matrix operations written in C for a significant speed up - assuming our matrices aren't too large to load in a single operation).

This is shown below (for clarity I have redeclared the parameters from above):

In [14]:
#imports
import numpy as np
import itertools

# define worker function that calculates result from input parameters
#  Note these must be in the same order as input to itertools.product
def science_result(mass, teff, lg, z):
    result = mass + teff**(-3) + lg/z
    # return result
    return result

# define ranges of parameters
Mass = np.arange(0.1, 2.1, 0.1)
Teff = np.arange(2000.0, 5500.0, 500.0)
logg = np.arange(3, 5.1, 0.05)
Z = np.arange(-1.5, 0.6, 0.1)

# get all combinations as a 1D list
combinations = list(itertools.product(Mass, Teff, logg, Z))

# convert to a numpy matrix
combinations = np.array(combinations)

# extract out our matrices (one could just use combinations)
Mass_m = combinations[:, 0]
Teff_m = combinations[:, 1]
logg_m = combinations[:, 2]
z_m = combinations[:, 3]

# work out all our results at once (no for loop required)
results = science_result(Mass_m, Teff_m, logg_m, z_m)

# or this:
# results = science_results(combinations[:, 0], combinations[:, 1],
#                           combinations[:, 2], combinations[:, 3])





